# COMP4318/5318 Assignment 2: Image Classification

### Group number: 91  , SID1: 500209446 , SID2: ...

This template notebook includes code to load the  dataset and a skeleton for the main sections that should be included in the notebook. Please stick to this struture for your submitted notebook.

Please focus on making your code clear, with appropriate variable names and whitespace. Include comments and markdown text to aid the readability of your code where relevant. See the specification and marking criteria in the associated specification to guide you when completing your implementation.

## Setup and dependencies
Please use this section to list and set up all your required libraries/dependencies and your plotting environment. 

In [15]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
import seaborn as sns


import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import time
# Make the notebook's output stable across runs
np.random.seed(42)
# Set up inline plotting and figure/axis labels

import matplotlib as mpl
import matplotlib.pyplot as plt
import tensorflow as tf
import keras_tuner
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras import layers
from keras.callbacks import EarlyStopping


## 1. Data loading, exploration, and preprocessing


Code to load the dataset is provided in the following cell. Please proceed with your data exploration and preprocessing in the remainder of this section.

In [22]:
# Load the dataset training and test sets as numpy arrays
# assuming Assignment2Data folder is present in the same directory 
# as the notebook
X_train = np.load('Assignment2Data/X_train.npy')
y_train = np.load('Assignment2Data/y_train.npy')
X_test = np.load('Assignment2Data/X_test.npy')
y_test = np.load('Assignment2Data/y_test.npy')

Number of training X: 18928 Y: 18928
Number of testing X: 4732 Y: 4732


In [4]:
classes = np.unique(y_train)
print("Class Labels: ", classes)
print(f"X_train shape : {X_train.shape}\ny_train shape : {y_train.shape}\nX_test shape : {X_test.shape}\ny_test shape : {y_test.shape}")

[ 0  1  2  3  4  5  6  7  8  9 10] [1741  936  916  978 1562 1651 4201 1519 1546 1682 2196]


### Examples of preprocessed data
Please print/display some examples of your preprocessed data here.

### Data Exploration

In [ ]:
class_dict = {}
actual_labels = ["bladder","femur-left","femur-right","heart","kidney-left","kidney-right","liver","lung-left","lung-right","pancreas","spleen"]
for i, num_label in enumerate(classes):
    class_dict[num_label]= actual_labels[i]
print("label dictionary :",class_dict)


fig, axes = plt.subplots(2,11, figsize=(17, 3))

for i, ax in enumerate(axes[0,:]):
    idx_tuple = np.where(y_train == classes[i])
    idx = idx_tuple[0][0]
    ax.imshow(X_train[idx], cmap='gray')
    ax.set_title(f"{classes[i]}: {actual_labels[classes[i]]}")
    ax.axis('off')
for i, ax in enumerate(axes[1,:]):
    idx_tuple = np.where(y_train == classes[i])
    idx = idx_tuple[0][1]
    ax.imshow(X_train[idx], cmap='gray')
    ax.set_title(f"{classes[i]}: {actual_labels[classes[i]]}")
    ax.axis('off')
plt.show()

unique, counts = np.unique(y_train, return_counts=True)
class_distribution = dict(zip(unique, counts))
print("Class distribution in training data:", class_distribution)

plt.figure(figsize=(5,3))
plt.bar(class_distribution.keys(), class_distribution.values(), color='lightseagreen')
plt.xlabel('Class Label')
plt.ylabel('Number of Examples')
plt.title('Class Distribution in Training Data')
plt.xticks(list(class_distribution.keys()))  # ensure all class labels are shown
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

### Data preprocessing

In [ ]:
def image_preprocessing(X_train, X_test):
    X_train = X_train / 255
    X_test = X_test / 255
    X_train = X_train.reshape(-1, 28*28) # number of rows required to keep the array's total size constant based on the other dimensions
    X_test = X_test.reshape(-1, 28*28)
    return X_train, X_test

def apply_pca(X_train, X_test, n_components=0.95):
    pca = PCA(n_components=n_components)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)
    print(X_train_pca.shape)
    print(X_test_pca.shape)
    return X_train_pca, X_test_pca
    
def expand_dimension(x_train,x_val,x_test):
    X_train_split = np.expand_dims(X_train_split, -1)
    X_val = np.expand_dims(X_val, -1)
    X_test = np.expand_dims(X_test, -1)
    return X_train_split,X_val,X_test

def plot_explained_variance(X_train_normalized):
    plt.figure(figsize=(8, 5))
    pca_model = PCA()
    X_train_pca = pca_model.fit_transform(X_train_normalized)
    explained_variance = np.cumsum(pca_model.explained_variance_ratio_)

    # Find the number of components to reach 95% variance
    num_components = np.argmax(explained_variance >= 0.95) + 1  # plus one because np.argmax returns zero-based index

    # Plot the explained variance and mark the point where it reaches 95%
    plt.plot(explained_variance, label='Cumulative Explained Variance')
    plt.scatter(num_components - 1, explained_variance[num_components - 1], color='red')  # Mark the 95% threshold point
    plt.axhline(y=0.95, color='r', linestyle='--', label='95% Explained Variance')
    plt.axvline(x=num_components - 1, color='r', linestyle='--')

    # Add labels and title
    plt.xlabel('Number of Components')
    plt.ylabel('Cumulative Explained Variance')
    plt.title('Explained Variance vs. Number of Components')
    plt.legend(loc='best')
    plt.grid(True)
    plt.show()



###  For the following cell :
###  1. PCA is not applied for MLP&CNN
###  2. train_test_split is used only for MLP&CNN ( SVM uses grid-search )

In [ ]:
# 1.  Normalization to range [0,1]
X_train_normalized, X_test_normalized = image_preprocessing(X_train, X_test)

# 2.  Apply Dimensionality reduction using PCA
X_train_pca, X_test_pca = apply_pca(X_train_normalized, X_test_normalized)

# 3. train and test spilt
X_train_split, X_val, y_train_split, y_val = train_test_split(X_train_normalized, y_train, stratify=y_train, train_size=0.8)

# 4. Input data for neural network  小鸡训练神经网络的变量🐤
X_train_split, X_val, X_test = expand_dimension(X_train_split, X_val, X_test_normalized)

# plot cumulative variance
plot_explained_variance(X_train_normalized)


## 2. Algorithm design and setup

### 2.1 Algorithm of choice from first six weeks of course - Support Vector Machine

In [ ]:
svm_model = svm.SVC(kernel='rbf', C=10, gamma=0.001)
svm_model.fit(X_train_pca, y_train)
y_pred = svm_model.predict(X_test_pca)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of the SVM on test data:", accuracy)
print(classification_report(y_test, y_pred))

### 2.2 Fully connected neural network

#### 2.2.1 Define MLP model

In [23]:
def build_mlp_model(hyperparameters):
    model = keras.Sequential()
    model.add(layers.Input(shape=(28,28,1)))
    model.add(layers.Flatten())

    hp_layers = hyperparameters.Int('layers', 1, 5)
    for _ in hp_layers:
        model.add(
            layers.Dense(
                # Define the hyperparameter.
                units=hyperparameters.Int("units", min_value=32, max_value=512, step=32),
                activation=hyperparameters.Choice("activation", ["relu", "tanh"]),
            )
        )
    model.add(layers.Dense(11, activation="softmax"))
    
    learning_rate = hyperparameters.Float("lr", min_value=1e-5, max_value=1e-3, sampling="log")
    optimizer=keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

In [24]:
mlp_model = keras.models.Sequential([
    keras.layers.Input(shape=(28,28,1)),
    keras.layers.Flatten(),
    keras.layers.Dense(300, activation="tanh"),
    keras.layers.Dense(100, activation="tanh"),
    keras.layers.Dense(11, activation="softmax")
])

opt = keras.optimizers.SGD(learning_rate=1e-4)
mlp_model.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

#### 2.2.2 Train MLP model

In [25]:
history = mlp_model.fit(X_train_split, y_train_split, epochs=100,
                    validation_data=(X_val, y_val))

Epoch 1/100
474/474 [==============================] - 1s 2ms/step - loss: 2.2797 - accuracy: 0.2221 - val_loss: 2.1263 - val_accuracy: 0.2644
Epoch 2/100
474/474 [==============================] - 1s 2ms/step - loss: 2.0420 - accuracy: 0.3001 - val_loss: 1.9883 - val_accuracy: 0.3228
Epoch 3/100
474/474 [==============================] - 1s 2ms/step - loss: 1.9179 - accuracy: 0.3618 - val_loss: 1.8807 - val_accuracy: 0.3735
Epoch 4/100
474/474 [==============================] - 1s 2ms/step - loss: 1.8228 - accuracy: 0.4014 - val_loss: 1.8010 - val_accuracy: 0.4068
Epoch 5/100
474/474 [==============================] - 1s 2ms/step - loss: 1.7538 - accuracy: 0.4292 - val_loss: 1.7456 - val_accuracy: 0.4245
Epoch 6/100
380/474 [=======================>......] - ETA: 0s - loss: 1.7047 - accuracy: 0.4504

KeyboardInterrupt: 

### Convolutional neural network

In [42]:
cnn_model = keras.Sequential([
    
    # Specify the input shape
    keras.Input(shape=(28, 28, 1)),
    
    # Conv and pool block 1
    keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    
    # Conv and pool block 2
    keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    
    # Flatten and classify using dense output layer
    keras.layers.Flatten(),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(11, activation="softmax"),
])

cnn_model.compile(loss='sparse_categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])
batch_size = 128
epochs = 10

In [43]:
history = cnn_model.fit(X_train_split, y_train_split, batch_size=batch_size,
                    epochs=epochs, validation_data=(X_val, y_val))

Epoch 1/10
119/119 [==============================] - 5s 40ms/step - loss: 3.0690 - accuracy: 0.4703 - val_loss: 1.0179 - val_accuracy: 0.6680
Epoch 2/10
119/119 [==============================] - 5s 39ms/step - loss: 0.9234 - accuracy: 0.6950 - val_loss: 0.6774 - val_accuracy: 0.7800
Epoch 3/10
119/119 [==============================] - 5s 40ms/step - loss: 0.6656 - accuracy: 0.7802 - val_loss: 0.5916 - val_accuracy: 0.8090
Epoch 4/10
119/119 [==============================] - 5s 40ms/step - loss: 0.5460 - accuracy: 0.8202 - val_loss: 0.5104 - val_accuracy: 0.8296
Epoch 5/10
119/119 [==============================] - 5s 39ms/step - loss: 0.4889 - accuracy: 0.8366 - val_loss: 0.4369 - val_accuracy: 0.8627
Epoch 6/10
119/119 [==============================] - 5s 39ms/step - loss: 0.4244 - accuracy: 0.8585 - val_loss: 0.4272 - val_accuracy: 0.8679
Epoch 7/10
119/119 [==============================] - 5s 40ms/step - loss: 0.3912 - accuracy: 0.8700 - val_loss: 0.4009 - val_accuracy: 0.8700

## 3. Hyperparameter tuning

### Algorithm of choice from first six weeks of course

In [ ]:
# use subset of data to do hyperparameter tuning
print(X_train_pca.shape)
X_train_pca = X_train_pca[:5000,:]
y_train = y_train[:5000]

param_grid = [
    {'kernel': ['poly'], 'C': [0.1, 1, 10], 'gamma': [0.001, 0.01], 'degree': [2, 3, 4]},
    {'kernel': ['rbf'], 'C': [0.1, 1, 10], 'gamma': [0.001, 0.01]}
]

svm_tuning = svm.SVC(class_weight='balanced')

grid_search = GridSearchCV(svm_tuning, param_grid, cv=5, scoring='accuracy', verbose=2)

grid_search.fit(X_train_pca, y_train)

print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))

In [ ]:
y_pred = grid_search.predict(X_test_pca)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of the best SVM on test data:", accuracy)
print(classification_report(y_test, y_pred))

### Fully connected neural network

In [28]:
def build_mlp_model(hyperparameters):
    model = keras.Sequential()
    model.add(layers.Input(shape=(28,28,1)))
    model.add(layers.Flatten())

    hp_layers = hyperparameters.Int('num_layers', 1, 5)
    for i in range(hp_layers):
        model.add(
            layers.Dense(
                # Define the hyperparameter.
                units=hyperparameters.Int(f"units_{i}", min_value=32, max_value=512, step=32),
                activation=hyperparameters.Choice("activation", ["relu", "tanh"]),
            )
        )
    
    if hyperparameters.Boolean("dropout"):
        model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(11, activation="softmax"))

    learning_rate = hyperparameters.Float("lr", min_value=1e-5, max_value=1e-3, sampling="log")
    optimizer=keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

In [12]:
build_mlp_model(keras_tuner.HyperParameters())
tuner = keras_tuner.BayesianOptimization(
    hypermodel=build_mlp_model,
    objective="val_accuracy",
    max_trials=100,
    overwrite=True,
    directory="mlp_tuning",
    project_name="mlp_tuning"
)
tuner.search_space_summary()

Search space summary
Default search space size: 5
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 5, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
dropout (Boolean)
{'default': False, 'conditions': []}
lr (Float)
{'default': 1e-05, 'conditions': [], 'min_value': 1e-05, 'max_value': 0.001, 'step': None, 'sampling': 'log'}


In [13]:
tuner.search(X_train_split, y_train_split, epochs=20, validation_data=(X_val, y_val))

Trial 100 Complete [00h 00m 30s]
val_accuracy: 0.8285789489746094

Best val_accuracy So Far: 0.8520866632461548
Total elapsed time: 01h 04m 38s


In [26]:
models = tuner.get_best_models(num_models=2)
best_model = models[0]
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 224)               175840    
                                                                 
 dense_1 (Dense)             (None, 480)               108000    
                                                                 
 dense_2 (Dense)             (None, 224)               107744    
                                                                 
 dense_3 (Dense)             (None, 416)               93600     
                                                                 
 dense_4 (Dense)             (None, 416)               173472    
                                                                 
 dense_5 (Dense)             (None, 11)                4

In [29]:
best_hps = tuner.get_best_hyperparameters(1)
print(best_hps[0])
model = build_mlp_model(best_hps[0])

usualCallback = EarlyStopping()
overfitCallback = EarlyStopping(monitor='loss', min_delta=0, patience = 10)

In [30]:
model.fit(x=X_train, y=y_train, epochs=100, callbacks=[overfitCallback])

Epoch 1/100
592/592 [==============================] - 3s 4ms/step - loss: 2.0621 - accuracy: 0.5775
Epoch 2/100
592/592 [==============================] - 2s 4ms/step - loss: 0.9185 - accuracy: 0.6989
Epoch 3/100
592/592 [==============================] - 2s 4ms/step - loss: 0.7667 - accuracy: 0.7460
Epoch 4/100
592/592 [==============================] - 2s 4ms/step - loss: 0.6723 - accuracy: 0.7762
Epoch 5/100
592/592 [==============================] - 2s 4ms/step - loss: 0.6087 - accuracy: 0.7964
Epoch 6/100
592/592 [==============================] - 2s 4ms/step - loss: 0.5648 - accuracy: 0.8107
Epoch 7/100
592/592 [==============================] - 2s 4ms/step - loss: 0.5294 - accuracy: 0.8200
Epoch 8/100
592/592 [==============================] - 2s 4ms/step - loss: 0.4701 - accuracy: 0.8391
Epoch 9/100
592/592 [==============================] - 2s 4ms/step - loss: 0.4669 - accuracy: 0.8409
Epoch 10/100
592/592 [==============================] - 2s 4ms/step - loss: 0.4145 - accura

### Convolutional neural network

## 4. Final models
In this section, please ensure to include cells to train each model with its best hyperparmater combination independently of the hyperparameter tuning cells, i.e. don't rely on the hyperparameter tuning cells having been run.

### Algorithm of choice from first six weeks of course

best_params = grid_search.best_params_
print(best_params)


best_svm_model = svm.SVC(C=best_params['C'],gamma=best_params['gamma'],kernel=best_params['kernel'],class_weight='balanced')
best_svm_model.fit(X_train_pca, y_train)
y_pred = best_svm_model.predict(X_test_pca)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of the best SVM on test data:", accuracy)
print(classification_report(y_test, y_pred))

In [ ]:
# Plot confusion matrix

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    plt.figure(figsize=(8, 6))
    sns_heatmap = sns.heatmap(cm, annot=True, fmt=".2f" if normalize else "d", cmap=cmap,
                xticklabels=classes, yticklabels=classes)
    plt.title(title)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

cm = confusion_matrix(y_test, y_pred)
plot_confusion_matrix(cm, classes=classes,
                      title='Confusion Matrix, without normalization')
# plot_confusion_matrix(cm, classes=classes, normalize=True,
#                       title='Normalized Confusion Matrix')


### Fully connected neural network

### Convolutional neural network